# Advanced Employee Turnover Prediction Notebook
This notebook contains the full pipeline with hyperparameter tuning and SMOTE.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import pickle
import warnings

warnings.filterwarnings('ignore')
RANDOM_STATE = 42


In [3]:
def load_data() -> pd.DataFrame:
    """Load turnover.csv from script directory or CWD."""
    script_dir = Path().resolve()
    cwd = Path.cwd().resolve()
    for location in (script_dir / "turnover.csv", cwd / "turnover.csv"):
        if location.exists():
            df = pd.read_csv(location, encoding="ISO-8859-1")
            print(f"Loaded {df.shape[0]}×{df.shape[1]} from {location}")
            return df
    raise FileNotFoundError("turnover.csv not found.")

def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['tenure_years'] = df['stag'] / 12.0
    df['age_group'] = pd.cut(df['age'], bins=[0,25,35,45,100], labels=['Young','Mid','Senior','Veteran'])
    df['personality_score'] = df[['extraversion','independ','selfcontrol','anxiety','novator']].mean(axis=1)
    df['stress_level'] = df['anxiety'] + df['selfcontrol']
    df['adaptability'] = df['novator'] + df['extraversion']
    df['tenure_age_ratio'] = df['tenure_years'] / df['age']
    df['coaching_impact'] = (df['coach']=='yes').astype(int) * df['personality_score']
    df['high_risk'] = ((df['stag']<12)&(df['age']<30)).astype(int)
    df['low_risk'] = ((df['stag']>60)&(df['age']>35)).astype(int)
    return df

def build_preprocessor(cat_cols, num_cols):
    return ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', StandardScaler(), num_cols)
    ])

def tune_classifier(pipeline, param_grid, X_train, y_train, name):
    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    print(f"Best {name} params: {grid.best_params_}")
    return grid.best_estimator_


In [5]:
# Load and prepare data
df = load_data()
df = feature_engineering(df)
y = df.pop('event')
X = df

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RANDOM_STATE)

# Define columns
cat_cols = ['gender','industry','profession','traffic','coach','head_gender','greywage','way','age_group']
num_cols = [c for c in X.columns if c not in cat_cols]

# Preprocessor
preproc = build_preprocessor(cat_cols, num_cols)

# Pipelines and parameter grids
rf_pipe = ImbPipeline([('prep', preproc), ('select', SelectKBest(f_classif, k=50)), ('smote', SMOTE(random_state=RANDOM_STATE)), ('clf', RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))])
rf_params = {'clf__n_estimators':[200,300], 'clf__max_depth':[10,15], 'clf__min_samples_split':[4,5], 'clf__min_samples_leaf':[1,2]}

gb_pipe = ImbPipeline([('prep', preproc), ('select', SelectKBest(f_classif, k=50)), ('smote', SMOTE(random_state=RANDOM_STATE)), ('clf', GradientBoostingClassifier(random_state=RANDOM_STATE))])
gb_params = {'clf__n_estimators':[100,200], 'clf__learning_rate':[0.05,0.1], 'clf__max_depth':[6,8], 'clf__min_samples_split':[3,5], 'clf__min_samples_leaf':[1,3]}

# Tune models
best_rf = tune_classifier(rf_pipe, rf_params, X_train, y_train, 'RandomForest')
best_gb = tune_classifier(gb_pipe, gb_params, X_train, y_train, 'GradientBoosting')

# Final ensemble
vote_clf = VotingClassifier(estimators=[('rf', best_rf.named_steps['clf']), ('gb', best_gb.named_steps['clf'])], voting='soft', n_jobs=-1)
final_pipe = ImbPipeline([('prep', preproc), ('select', SelectKBest(f_classif, k=50)), ('smote', SMOTE(random_state=RANDOM_STATE)), ('vote', vote_clf)])
final_pipe.fit(X_train, y_train)

# Evaluate
y_pred = final_pipe.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('ROC AUC:', roc_auc_score(y_test, final_pipe.predict_proba(X_test)[:,1]))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Save model
with open('employee_turnover_optimized.pkl','wb') as f:
    pickle.dump(final_pipe, f)


Loaded 1129×16 from C:\Users\Rajiv Arora\Desktop\EmployeeTurnover\turnover.csv
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best RandomForest params: {'clf__max_depth': 15, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 5, 'clf__n_estimators': 300}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best GradientBoosting params: {'clf__learning_rate': 0.05, 'clf__max_depth': 6, 'clf__min_samples_leaf': 3, 'clf__min_samples_split': 3, 'clf__n_estimators': 200}
Accuracy: 0.6460176991150443
ROC AUC: 0.7291666666666667
              precision    recall  f1-score   support

           0       0.65      0.61      0.63       112
           1       0.64      0.68      0.66       114

    accuracy                           0.65       226
   macro avg       0.65      0.65      0.65       226
weighted avg       0.65      0.65      0.65       226

[[68 44]
 [36 78]]
